In [64]:
# This script takes in two xlsx files and returns two new updated xlsx files wihtout changing the original files.
# The only new info to add before running is the text you want added to the notes. This has the following for the
# the example
new_text_for_notes = "2.8.2022-br DOS updated for trial"


In [65]:
import pandas as pd

In [66]:
# Convert the Excel files into panda dataframes #new_dos = 'New_DOS_file.xlsx' #master = 'MasterFile.xlsx'

dos_df = pd.read_excel('TrialDOSUpdate.xlsx', header=0, index_col=False, keep_default_na=True)
master_df = pd.read_excel('TrialMaster.xlsx', header=0, index_col=False, keep_default_na=True)


In [68]:
print(dos_df)
print(master_df)

    chartID  Begin DOS    End DOS
0      1558 2021-03-03 2021-03-05
1      1558 2021-03-06 2021-03-06
2      1558 2021-03-07 2021-03-07
3      1558 2021-03-08 2021-03-08
4      1558 2021-03-09 2021-03-09
5      1129 2021-03-10 2021-03-10
6      1129 2021-03-11 2021-03-11
7      1129 2021-03-12 2021-03-12
8      1212 2021-03-13 2021-03-13
9      1745 2021-03-14 2021-03-14
10     1745 2021-03-15 2021-03-15
11     1745 2021-03-16 2021-03-16
12     1745 2021-03-17 2021-03-17
13     1745 2021-03-18 2021-03-18
14     1745 2021-03-19 2021-03-19
15     1745 2021-03-20 2021-03-20
16     1745 2021-03-21 2021-03-21
17     1745 2021-03-22 2021-03-22
18     1745 2021-03-23 2021-03-23
19     1745 2021-03-24 2021-03-24
20     1745 2021-03-25 2021-03-25
21     1745 2021-03-26 2021-03-26
22     1745 2021-03-27 2021-03-27
23     1745 2021-03-28 2021-03-28
24     1745 2021-03-29 2021-03-29
25     1745 2021-03-30 2021-03-30
26     1745 2021-03-31 2021-03-31
27     1745 2021-04-01 2021-04-01
28     1745 20

In [69]:
# Need to convert to dates datatypes. I will also create a new column called rangeDOS for those requests that have 
# info in the Begin and End DOSs that need to be caputred. This is also where I format the cells.
dos_df['Begin DOS'] = dos_df['Begin DOS'].dt.strftime('%m/%d/%Y')
dos_df['End DOS'] = dos_df['End DOS'].dt.strftime('%m/%d/%Y')
master_df['Begin DOS'] = master_df['Begin DOS'].dt.strftime('%m/%d/%Y')

master_df['Begin DOS'] = master_df['Begin DOS'].astype(str)
dos_df['Begin DOS'] = dos_df['Begin DOS'].astype(str)
dos_df['End DOS'] = dos_df['End DOS'].astype(str)
dos_df['Range DOS'] = dos_df[['Begin DOS', 'End DOS']].agg('-'.join, axis=1)



In [70]:
print(dos_df['Begin DOS'].head())
print(type(dos_df['Begin DOS']))
print(dos_df["Range DOS"].head())

0    03/03/2021
1    03/06/2021
2    03/07/2021
3    03/08/2021
4    03/09/2021
Name: Begin DOS, dtype: object
<class 'pandas.core.series.Series'>
0    03/03/2021-03/05/2021
1    03/06/2021-03/06/2021
2    03/07/2021-03/07/2021
3    03/08/2021-03/08/2021
4    03/09/2021-03/09/2021
Name: Range DOS, dtype: object


In [71]:
# Compare the begin and end DOS. By using the numpy select and not just where, we could add multiple conditions.
# The result will be a new column that either has the Begin DOS or the Range DOS; this column will then be used 
# to create the concat list to add to the master.
#conditions = [(dos_df['Begin DOS'] is dos_df['End DOS']), (dos_df['Begin DOS'] is not dos_df['End DOS'])]
#choices = [dos_df['Begin DOS'], dos_df['Range DOS']]
#dos_df['DOS to concat'] = np.select(conditions, choices, default=np.nan)

for i in range(len(dos_df)):
    if dos_df.loc[i,"Begin DOS"] == dos_df.loc[i,"End DOS"]:
        dos_df.loc[i,"DOS to concat"] = dos_df.loc[i,"Begin DOS"]
    else:
        dos_df.loc[i,"DOS to concat"] = dos_df.loc[i,"Range DOS"]

In [72]:
print(dos_df.iloc[0])
print(dos_df.iloc[1])

chartID                           1558
Begin DOS                   03/03/2021
End DOS                     03/05/2021
Range DOS        03/03/2021-03/05/2021
DOS to concat    03/03/2021-03/05/2021
Name: 0, dtype: object
chartID                           1558
Begin DOS                   03/06/2021
End DOS                     03/06/2021
Range DOS        03/06/2021-03/06/2021
DOS to concat               03/06/2021
Name: 1, dtype: object


In [73]:
#This step takes the dataframe and groups by the ChartID, making a list out of the all the 
# dates in the dos column into a list. We then want to drop any duplicates.
grouped_chartIDs = dos_df.groupby(by='chartID', axis=0)['DOS to concat'].apply(lambda x : ', '.join(x)).reset_index()
dos_to_add = grouped_chartIDs.drop_duplicates()


In [74]:
print(type(dos_to_add))
print(type(grouped_chartIDs))
print(dos_to_add)


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   chartID                                      DOS to concat
0     1129                 03/10/2021, 03/11/2021, 03/12/2021
1     1212                                         03/13/2021
2     1558  03/03/2021-03/05/2021, 03/06/2021, 03/07/2021,...
3     1745  03/14/2021, 03/15/2021, 03/16/2021, 03/17/2021...


In [75]:
# Now we want to export an excel
dos_to_add.to_excel('newdosfile.xlsx', index=False)

In [76]:
# Now we want to replace the DOS in the master with the updated DOSs
#master_df['Begin DOS'] = master_df['chartID'].map(dos_to_add['chartID'],dos_to_add['DOS to concat'])
for i in range(len(master_df)):
    for x in range(len(dos_to_add)):
        if dos_to_add.loc[x,"chartID"] == master_df.loc[i,"chartID"]:
            master_df.loc[i,"Begin DOS"] = dos_to_add.loc[x,"DOS to concat"]
            master_df.loc[i,"Notes"] = new_text_for_notes + "; " + master_df.loc[i,"Notes"]
        else:
            master_df.loc[i,"Begin DOS"] = master_df.loc[i,"Begin DOS"]

In [77]:
print(master_df)
#master_df = master_df.reset_index(drop=True, inplace=True)
print(type(master_df))


     Dummy Row  chartID                                          Begin DOS  \
0           Do     1129                 03/10/2021, 03/11/2021, 03/12/2021   
1          not     1212                                         03/13/2021   
2       change     1745  03/14/2021, 03/15/2021, 03/16/2021, 03/17/2021...   
3         this     1558  03/03/2021-03/05/2021, 03/06/2021, 03/07/2021,...   
4          row     1244                                         10/01/2021   
5  or this one    21345                                         11/01/2021   

  Dummy Row2                                         Notes  
0         Or  2.8.2022-br DOS updated for trial; overwrite  
1       this  2.8.2022-br DOS updated for trial; overwrite  
2        one  2.8.2022-br DOS updated for trial; OvERWrite  
3      don't  2.8.2022-br DOS updated for trial; OvErwrite  
4      touch            dummy column shouldn't be replaced  
5         it            dummy column shouldn't be replaced  
<class 'pandas.core.frame.

In [78]:
# Now we want to export a new master excel file
master_df.to_excel('NewMaster.xlsx', index=False)